In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Dissertation/data/pickled_for_colab.csv",encoding='latin-1')

In [7]:
! wget http://nlp.stanford.edu/data/glove.6B.zip
! unzip glove*.zip

--2021-12-16 09:26:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-16 09:26:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-16 09:26:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
#for trans in df['transcript']:
haha = ["When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest. Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment."]
return_trans = []

for trans in df['transcript']:
  sentences = sent_tokenize(trans)
  #sentences = [sent.lower() for sent in sentences]
  vectors = []
  for sent in sentences:
    w_lst = sent.split()
    v = sum([embeddings.get(w, np.zeros((100,))) for w in w_lst])
    v = v/ (len(w_lst) + 0.00001)
    vectors.append(v)
  matrix = np.ones([len(sentences), len(sentences)])
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      matrix[i][j] = cosine_similarity(vectors[i].reshape(1, 100), vectors[j].reshape(1, 100))[0, 0]
  graph = nx.from_numpy_array(matrix)
  scores = nx.pagerank(graph, alpha=0.85, max_iter=100)
  sentence_scores = ((scores[i], s) for i,s in enumerate(sentences))
  ranked_sentences = [ranked for (_, ranked) in sorted(sentence_scores, reverse=True)]
  ret = ' '.join(sent for sent in ranked_sentences)
  return_trans.append(ret)

df_ret = DataFrame(return_trans,columns=['transcript'])
df_ret.to_csv('textrank_trans.csv', index = False, header = True)



KeyboardInterrupt: ignored

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [10]:
embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    vals = line.split()
    word = vals[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()